# Case 1

## Generate Data

In [2]:
# first_name, last_name, age, country
from faker import Faker
import pandas as pd
from random import randint

faker = Faker()
countries = [
    "US", "UK", "ID", "SG", "MA",
    "TO", "GH", "IS", "ER", "BB"
]

number_of_data = 100000
dummy_data = []

for i in range(number_of_data):
    name = faker.name().split()
    first_name, last_name = name[0], name[-1]
    age = randint(10, 60)
    country = countries[randint(0, 9)]

    dummy_data.append({
        "first_name": first_name,
        "last_name": last_name,
        "age": str(age),
        "country": country
    })

df = pd.DataFrame(dummy_data)

df.head()


,first_name,last_name,age,country
0,John,Jones,52,GH
1,James,Moore,57,MA
2,Victor,King,49,GH
3,John,Beck,15,IS
4,Elizabeth,Freeman,30,US


In [3]:
import numpy as np
from random import random

messy_df = df.copy()

# Set the probability of a None value
missing_rate = 0.2 

# Function to randomly insert None
def insert_missing_values(df, column, rate, add_unknown_and_random_str=False):
    mask = np.random.rand(len(df)) < rate
    if add_unknown_and_random_str:
        prob = random()
        if prob < 0.5:
            df.loc[mask, column] = "Unknown"
        else:
            df.loc[mask, column] = "cfasew"
    else:
        df.loc[mask, column] = None

# Apply to desired columns
for col in ['first_name', 'last_name', 'age', 'country']:
    insert_missing_values(messy_df, col, missing_rate)

for col in ['age', 'country']:
    insert_missing_values(messy_df, col, missing_rate, add_unknown_and_random_str=True)

messy_df.head(10)

,first_name,last_name,age,country
0,None,Jones,52,Unknown
1,James,Moore,57,Unknown
2,Victor,King,49,GH
3,John,Beck,15,IS
4,Elizabeth,Freeman,None,US
5,Philip,None,13,None
6,James,Pruitt,None,BB
7,Miguel,None,58,None
8,None,Rocha,22,Unknown
9,Ryan,Gomez,12,SG


## Original Script

In [4]:
import pandas as pd

def clean_customer_data(df):
    known_countries = ["US", "UK", "ID", "SG"]
    full_names = []
    cleaned_ages = []
    is_adults = []
    standardized_countries = []

    for i in range(len(df)):
        row = df.iloc[i]

        # String concat with NaN checking
        first = str(row['first_name']) if pd.notna(row['first_name']) else "Unknown"
        last = str(row['last_name']) if pd.notna(row['last_name']) else "Unknown"
        full_name = first + " " + last
        full_names.append(full_name)

        # Data type conversion & filtering
        age_raw = row['age']
        if pd.isna(age_raw) or str(age_raw).strip().lower() == 'unknown':
            age = -1
        else:
            try:
                age = int(age_raw)
            except:
                age = -1
        cleaned_ages.append(age)

        # NA + conditional logic
        is_adults.append(True if age >= 18 else False)

        # String cleaning & filtering
        country = str(row['country']).strip().upper() if pd.notna(row['country']) else "XX"
        if country not in known_countries:
            country = "XX"
        standardized_countries.append(country)

    # Add results back to DataFrame
    df['full_name'] = full_names
    df['cleaned_age'] = cleaned_ages
    df['is_adult'] = is_adults
    df['country_code'] = standardized_countries

    return df

In [5]:
import timeit

df1 = messy_df.copy()
time_taken = timeit.timeit(lambda: clean_customer_data(df1), number=1)
print(f"Time taken: {time_taken:.4f} seconds")

Time taken: 4.7153 seconds


## Optimized Script

In [6]:
known_countries = ["US", "UK", "ID", "SG"]

def clean_customer_name(name):
    if pd.isna(name):
        return "Unknown"
    
    return name

def clean_customer_age(age):
    if pd.isna(age) or str(age).strip().lower() == "unknown":
        return -1
    
    try:
        return int(age)
    except:
        return -1

def is_customer_adult(age):
    return age >= 18

def clean_customer_country(country):
    if country not in known_countries or country is None:
        return  "XX"

    return country

def optimized_clean_customer_data(df):
    df["full_name"] = df["first_name"].apply(clean_customer_name) + " " + df["last_name"].apply(clean_customer_name)
    df["cleaned_age"] = df["age"].apply(clean_customer_age)
    df["is_adult"] = df["cleaned_age"].apply(is_customer_adult)
    df["country_code"] = df["country"].apply(clean_customer_country)

    return df

In [7]:
df2 = messy_df.copy()

time_taken = timeit.timeit(lambda: optimized_clean_customer_data(df2), number=1)
print(f"Time taken: {time_taken:.4f} seconds")

Time taken: 0.2099 seconds


In [8]:
df1.equals(df2)

True

# Case 2

## Generate Data

In [118]:
# entry_time, exit_time, break_duration, depratment
from random import randint
import pandas as pd

possible_departments = [
    "Sales", "Logistics", "Data", "Marketing", "Accounting",
    "Finance", "IT", "Product" 
]

number_of_data = 100000
dummy_data = []
for i in range(number_of_data):
    dummy_data.append({
        "entry_time": f"{randint(8, 10):02}:{randint(0, 59):02}:{randint(0, 59):02}",
        "exit_time": f"{randint(16, 18):02}:{randint(0, 59):02}:{randint(0, 59):02}",
        "break_duration": f"{randint(30, 90)}",
        "department": possible_departments[randint(0, 7)]
    })

df = pd.DataFrame(dummy_data)
df.head()

,entry_time,exit_time,break_duration,department
0,09:49:05,17:48:28,33,Accounting
1,09:08:13,18:15:04,60,IT
2,09:45:19,18:55:01,56,Accounting
3,08:45:57,18:12:44,37,Marketing
4,09:32:22,17:28:45,79,Finance


In [124]:
import numpy as np
from random import random

messy_df = df.copy()

# Set the probability of a None value
missing_rate = 0.2 

# Function to randomly insert None
def insert_missing_values(df, column, rate, add_unknown_and_random_str=False):
    mask = np.random.rand(len(df)) < rate
    if add_unknown_and_random_str:
        prob = random()
        if prob < 0.5:
            df.loc[mask, column] = "Unknown"
        else:
            df.loc[mask, column] = "csda"
    else:
        df.loc[mask, column] = None

# Apply to desired columns
for col in ['entry_time', 'exit_time', 'break_duration', 'department']:
    insert_missing_values(messy_df, col, missing_rate)

for col in ['entry_time', 'exit_time', 'break_duration', 'department']:
    insert_missing_values(messy_df, col, missing_rate, add_unknown_and_random_str=True)

messy_df.head(10)

,entry_time,exit_time,break_duration,department
0,09:49:05,17:48:28,33,Unknown
1,09:08:13,18:15:04,60,IT
2,09:45:19,18:55:01,None,Accounting
3,Unknown,18:12:44,37,Unknown
4,09:32:22,17:28:45,None,Finance
5,None,16:34:34,None,Data
6,08:04:53,16:21:31,csda,Data
7,Unknown,17:38:12,None,None
8,08:27:04,18:10:49,None,None
9,None,16:56:49,50,None


## Original Script

In [125]:
from datetime import datetime, timedelta

def preprocess_time_entries(df):
    work_durations = []
    work_day_labels = []
    departments = []
    
    for i in range(len(df)):
        row = df.iloc[i]
        
        # Clean entry_time and exit_time, convert to datetime
        try:
            entry_time = datetime.strptime(str(row['entry_time']), '%H:%M:%S')
        except ValueError:
            entry_time = None
        
        try:
            exit_time = datetime.strptime(str(row['exit_time']), '%H:%M:%S')
        except ValueError:
            exit_time = None
        
        # Handle break_duration, converting to minutes or setting to 0 if invalid
        try:
            break_duration = int(row['break_duration'])
        except (ValueError, TypeError):
            break_duration = 0
        
        # Calculate work duration in seconds, then convert to HH:MM:SS
        if entry_time and exit_time:
            total_seconds = (exit_time - entry_time).seconds - (break_duration * 60)
            work_duration = str(timedelta(seconds=total_seconds))
            work_durations.append(work_duration)
        else:
            work_durations.append('Invalid Time')
        
        # Assign work day labels based on work duration
        if entry_time and exit_time:
            total_hours = (exit_time - entry_time).seconds / 3600 - break_duration / 60
            if total_hours < 5:
                work_day_labels.append('Short Day')
            elif total_hours >= 8:
                work_day_labels.append('Long Day')
            else:
                work_day_labels.append('Regular Day')
        else:
            work_day_labels.append('Invalid Entry')
        
        # Clean department column
        department = row['department'] if pd.notna(row['department']) and row['department'].strip() != "" else "Unknown"
        departments.append(department)
    
    # Add results back to the DataFrame
    df['work_duration'] = work_durations
    df['work_day_label'] = work_day_labels
    df['department'] = departments
    
    return df

In [126]:
df1 = messy_df.copy()

time_taken = timeit.timeit(lambda: preprocess_time_entries(df1), number=1)
print(f"Time taken: {time_taken:.4f} seconds")

Time taken: 6.7860 seconds


In [266]:
def calculate_work_duration(entry_time, exit_time, break_duration):
    total_seconds = (exit_time - entry_time).dt.seconds - (break_duration * 60)
    work_duration = pd.to_timedelta(total_seconds, unit="s")

    return work_duration
    
    
def label_work_duration(total_hours):
    if pd.isna(total_hours):
        return "Invalid Entry"
    
    else:
        if total_hours < 5:
            return 'Short Day'
        elif total_hours >= 8:
            return 'Long Day'
        else:
            return 'Regular Day'

def clean_deparment(department):
    if pd.notna(department) and department.strip() != "":
        return department
    else:
        return "Unknown"
    
def optimized_preprocess_time_entries(df):
    entry_time = pd.to_datetime(df["entry_time"], format="%H:%M:%S", errors="coerce")
    exit_time = pd.to_datetime(df["exit_time"], format="%H:%M:%S", errors="coerce")
    break_duration = pd.to_numeric(df["break_duration"], "coerce").fillna(0)
    
    df["work_duration"] = calculate_work_duration(entry_time, exit_time, break_duration)

    work_duration_hour = df["work_duration"].dt.seconds/3600
    
    df["work_day_label"] = work_duration_hour.apply(label_work_duration)
    df["department"] = df["department"].apply(clean_deparment)

    df["work_duration"] = df["work_duration"].astype(str).str.replace(r"0 days 0|0 days", "", regex=True).str.strip()
    df["work_duration"] = df["work_duration"].str.replace("NaT", "Invalid Time")

In [267]:
df2 = messy_df.copy()

time_taken = timeit.timeit(lambda: optimized_preprocess_time_entries(df2), number=1)
print(f"Time taken: {time_taken:.4f} seconds")

Time taken: 0.8935 seconds


In [268]:
df1

,entry_time,exit_time,break_duration,department,work_duration,work_day_label
0,09:49:05,17:48:28,33,Unknown,7:26:23,Regular Day
1,09:08:13,18:15:04,60,IT,8:06:51,Long Day
2,09:45:19,18:55:01,None,Accounting,9:09:42,Long Day
3,Unknown,18:12:44,37,Unknown,Invalid Time,Invalid Entry
4,09:32:22,17:28:45,None,Finance,7:56:23,Regular Day
...,...,...,...,...,...,...
99995,09:27:07,17:43:25,61,Unknown,7:15:18,Regular Day
99996,09:07:20,17:48:13,None,Unknown,8:40:53,Long Day
99997,10:54:18,None,65,Unknown,Invalid Time,Invalid Entry
99998,Unknown,csda,32,Sales,Invalid Time,Invalid Entry


In [269]:
df1.equals(df2)

True

# Case 3

## Generate Data

In [125]:
# sales_amount, sale_date, sales_rep, products_sold, discount_applied
from faker import Faker
from random import random, randint, sample
from datetime import datetime
import pandas as pd

start_date = datetime.strptime("2023-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2025-01-01", "%Y-%m-%d")

faker = Faker()

faker.random_element()
possible_products = [
    "Television", "Refrigerator", "Handphone", "Camera", "Parfume",
    "Skin Care", "Keyboard", "Mouse", "Books", "Smart Watch"
]
sales_reps = [faker.name() for i in range(6000)]

number_of_data = 100000
dummy_data = []
for i in range(number_of_data):
    dummy_data.append({
        "sales_amount": f"USD {random()*randint(100, 2500)}",
        "sale_date": faker.date_between(start_date, end_date).strftime("%Y-%m-%d"),
        "sales_rep": sample(sales_reps, 1)[0],
        "products_sold": ", ".join(sample(possible_products, randint(1, 7))),
        "discount_applied": "Yes" if random() > 0.5 else "No"
    })

df = pd.DataFrame(dummy_data)
df.head()

,sales_amount,sale_date,sales_rep,products_sold,discount_applied
0,USD 252.2048908788015,2023-05-15,Sandra Thompson,"Books, Camera, Smart Watch",No
1,USD 944.7347605878658,2024-04-12,Katie Johnson,"Refrigerator, Television, Books, Handphone",Yes
2,USD 417.1940464382603,2024-01-30,Adrian Hunt,"Smart Watch, Keyboard, Television, Mouse, Hand...",No
3,USD 12.348894118463507,2024-04-25,Carmen Hall,"Keyboard, Television, Refrigerator, Handphone,...",Yes
4,USD 1495.2810359107177,2023-07-28,Matthew Spence,"Handphone, Smart Watch, Refrigerator, Camera, ...",No


## Original Script

In [126]:
def preprocess_sales_data(df):
    # Clean the 'sales_amount' column, remove currency symbols and convert to float
    sales_amounts = []
    for amount in df['sales_amount']:
        cleaned_amount = ''.join(c for c in amount if c.isdigit() or c == '.')
        try:
            sales_amounts.append(float(cleaned_amount))
        except ValueError:
            sales_amounts.append(0.0)
    
    df['sales_amount'] = sales_amounts
    
    # Convert 'sale_date' to datetime format
    df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
    
    # Calculate total sales per sales rep
    total_sales_per_rep = df.groupby('sales_rep')['sales_amount'].sum()
    df['total_sales'] = df['sales_rep'].map(total_sales_per_rep)
    
    # Aggregate unique products sold by each sales rep
    unique_products_per_rep = df['products_sold'].str.split(',').apply(lambda x: set(x))
    products_per_rep = unique_products_per_rep.groupby(df['sales_rep']).apply(lambda x: set([item for sublist in x for item in sublist]))
    df['unique_products'] = df['sales_rep'].map(products_per_rep)
    
    # Calculate the average sales amount per sales rep
    avg_sales_per_rep = df.groupby('sales_rep')['sales_amount'].mean()
    df['average_sales_amount'] = df['sales_rep'].map(avg_sales_per_rep)
    
    # Count the number of sales where a discount was applied
    discount_counts = df.groupby('sales_rep')['discount_applied'].apply(lambda x: (x == 'Yes').sum())
    df['discount_count'] = df['sales_rep'].map(discount_counts)
    
    return df


In [127]:
import timeit

df1 = df.copy()

time_taken = timeit.timeit(lambda: preprocess_sales_data(df1), number=1)
print(f"Time taken: {time_taken:.4f} seconds")

Time taken: 1.3260 seconds


## Optimized Script

In [133]:
import re
from collections import defaultdict

def clean_sales_amount(sales_amount):
    match = re.search(r"[A-Z]{3}\s([\d,]+\.\d+)", sales_amount)

    if match:
        amount = float(match.group(1).replace(",", ""))
    else:
        amount = 0.0

    return amount

def optimized_preprocess_sales_date(df):
    df["sales_amount"] = df["sales_amount"].apply(clean_sales_amount)

    # Convert 'sale_date' to datetime format
    df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
    
    # Calculate total sales per sales rep
    total_sales_per_rep = df.groupby('sales_rep')['sales_amount'].sum()
    df['total_sales'] = df['sales_rep'].map(total_sales_per_rep)
    
    # Aggregate unique products sold by each sales rep
    products_per_rep = defaultdict(set)
    for rep, products in zip(df['sales_rep'], df['products_sold']):
        for product in products.split(','):
            products_per_rep[rep].add(product)
            
    df['unique_products'] = df['sales_rep'].map(products_per_rep)
    
    # Calculate the average sales amount per sales rep
    avg_sales_per_rep = df.groupby('sales_rep')['sales_amount'].mean()
    df['average_sales_amount'] = df['sales_rep'].map(avg_sales_per_rep)
    
    # Count the number of sales where a discount was applied
    discount_counts = df.groupby('sales_rep')['discount_applied'].apply(lambda x: (x == 'Yes').sum())
    df['discount_count'] = df['sales_rep'].map(discount_counts)
    
    return df

In [135]:
import timeit

df2 = df.copy()

time_taken = timeit.timeit(lambda: optimized_preprocess_sales_date(df2), number=1)
print(f"Time taken: {time_taken:.4f} seconds")

Time taken: 0.9839 seconds
